In [1]:
import ray 

ray.init(num_gpus=1)

2022-11-13 13:41:19,278	INFO worker.py:1528 -- Started a local Ray instance.


Python version:,3.8.13
Ray version:,2.1.0


In [3]:
import ray
config = {
    "env": "Pendulum-v1",
    "num_gpus":1,
}
stop = {"episode_reward_mean": -1000}
ray.shutdown()
ray.init(
  num_gpus=1,
  include_dashboard=False,
  ignore_reinit_error=True,
  log_to_driver=False,
)
# execute training 
analysis = ray.tune.run(
  "SAC",
  config=config,
  stop=stop,
  checkpoint_at_end=True,
  local_dir="Pendulum_v1"  
)

2022-11-13 13:54:18,665	INFO worker.py:1528 -- Started a local Ray instance.


2022-11-13 14:04:55,786	INFO tune.py:777 -- Total run time: 636.35 seconds (635.35 seconds for the tuning loop).


In [9]:
import ray.rllib.algorithms.sac as sac
# restore a trainer from the last checkpoint
trial = analysis.get_best_logdir("episode_reward_mean", "max")
checkpoint = analysis.get_best_checkpoint(
  trial,
  "training_iteration",
  "max",
)
trainer = sac.SAC(config=config)
trainer.restore(checkpoint)

2022-11-13 14:23:57,382	WARNING util.py:66 -- Install gputil for GPU system monitoring.
2022-11-13 14:23:57,439	INFO trainable.py:766 -- Restored on 127.0.0.1 from checkpoint: /Users/davidkwak/tensorflow-test/gym_env/classic_control/Pendulum_v1/SAC/SAC_Pendulum-v1_f6252_00000_0_2022-11-13_13-54-19/checkpoint_000148
2022-11-13 14:23:57,440	INFO trainable.py:775 -- Current state after restoring: {'_iteration': 148, '_timesteps_total': None, '_time_total': 620.2296693325043, '_episodes_total': 78}


In [12]:
import gym

env = gym.make("Pendulum-v1")
obs = env.reset()

while True:
    action = trainer.compute_action(obs)
    obs,reward,done, _ = env.step(action)
    env.render()
    if done:
        break
env.close()

In [13]:
from gym.wrappers import RecordVideo

env = RecordVideo(gym.make("Pendulum-v1"),"sac_video")
obs = env.reset()

while True:
    action = trainer.compute_action(obs)
    obs,reward,done, _ = env.step(action)
    env.render()
    if done:
        break
env.close()

E1113 14:26:02.952237000 8116724992 fork_posix.cc:76]                  Other threads are currently calling into gRPC, skipping fork() handlers
E1113 14:26:02.975362000 8116724992 fork_posix.cc:76]                  Other threads are currently calling into gRPC, skipping fork() handlers
